In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as opt

import plotly.express as px
import zipfile

NA = 9999

In [2]:
census_path = "input/RP2019_INDCVIZA_csv.zip"
output_path = "output"

### Configuration

In [3]:
household_size_target_means = {
    "S1": 3.093, "S2": 2.938, "S3": 3.402, "S4": 2.783
}

number_of_cars_target_means = {
    "S1": 1.135, "S2": 1.027, "S3": 1.027, "S4": 1.081
}

location_type_factors = {
    "S1": {},
    "S2": { 1: 0.04, 3: -0.04 },
    "S3": { 1: 0.04, 3: -0.04 },
    "S4": { 1: -0.04, 3: 0.04 }
}

### Preparing the data

In [4]:
with zipfile.ZipFile(census_path) as archive:
    with archive.open("FD_INDCVIZA_2019.csv") as f:
        df_census = pd.read_csv(f, sep = ";", usecols = [
            "VOIT", "INPER", "IPONDI", "ILT", "ILETUD", "ETUD", "TACT"
        ], dtype = {
            "ILETUD": str, "ILT": str, "INPER": str, "IPONDI": float, "VOIT": str,
            "ETUD": str, "TACT": str
        })

In [5]:
# Clean weight
df_census["weight"] = df_census["IPONDI"]

In [6]:
# Clean household size
df_census["household_size"] = pd.to_numeric(df_census["INPER"], errors = "coerce").fillna(NA).astype(int)

In [7]:
df_plot = df_census.groupby("household_size")["weight"].sum().reset_index()
df_plot = df_plot.sort_values(by = "household_size")
df_plot["household_size"] = df_plot["household_size"].astype(str).replace({ str(NA): "NA" })
px.bar(df_plot, x = "household_size", y = "weight", title = "Household size: initial distribution")

In [8]:
# Clean number of cars
df_census["number_of_cars"] = pd.to_numeric(df_census["VOIT"], errors = "coerce").fillna(NA).astype(int)

In [9]:
df_plot = df_census.groupby("number_of_cars")["weight"].sum().reset_index()
df_plot = df_plot.sort_values(by = "number_of_cars")
df_plot["number_of_cars"] = df_plot["number_of_cars"].astype(str).replace({ str(NA): "NA" })
px.bar(df_plot, x = "number_of_cars", y = "weight", title = "Number of cars: initial distribution")

In [10]:
# Take the larger one between the two ?

# Clean location type
df_census["studies"] = df_census["ETUD"] == "1"
df_census["employed"] = df_census["TACT"] == "11"

# ... location type of employed persons
df_census.loc[df_census["employed"],
    "location_type"] = df_census.loc[df_census["employed"], "ILT"]

# ... location type of studying persons
df_census.loc[df_census["studies"], 
    "location_type"] = df_census.loc[df_census["studies"], "ILETUD"]

# ... cleanup
df_census.loc[~df_census["location_type"].isin(["1", "2", "3"]), "location_type"] = str(NA)
df_census["location_type"] = df_census["location_type"].astype(int)

In [11]:
df_plot = df_census.groupby("location_type")["weight"].sum().reset_index()
df_plot = df_plot.sort_values(by = "location_type")
df_plot["location_type"] = df_plot["location_type"].astype(str).replace({ str(NA): "NA" })
px.bar(df_plot, x = "location_type", y = "weight", title = "Location type: initial distribution")

In [12]:
df_density = df_census.groupby("location_type")["weight"].sum().reset_index()
df_density["density"] = df_density["weight"] / df_density["weight"].sum()
df_density["location_type"] = df_density["location_type"].replace({ NA: "NA" })
df_density[["location_type", "density"]]

,location_type,density
0,1,0.294980
1,2,0.152992
2,3,0.225308
3,NA,0.326720


<font color="red">Not exactly the same, but roughly ok!</font>

### Calculating the baseline means

In [13]:
# Household size
f = df_census["household_size"] != NA

mean = (df_census.loc[f]["household_size"] * df_census.loc[f]["weight"]).sum() / df_census.loc[f]["weight"].sum()
print("Mean household size:", mean)

Mean household size: 3.1387216830289852


<font color="red">A bit different from Tjark. Why?</font>

In [14]:
# Household size
f = df_census["number_of_cars"] != NA

mean = (df_census.loc[f, "number_of_cars"] * df_census.loc[f, "weight"]).sum() / df_census.loc[f, "weight"].sum()
print("Mean number of cars:", mean)

Mean number of cars: 1.0810140047942078


<font color="green">Exactly like Tjark!</font>

### Distribution calibration

Assume an attribute with $N$ unique values $v_i$ sorted in ascending order ($v_i \leq v_{i+1}$). The observation share for each attribute is $f_i$ (with $\sum f_i = 1$ and $f_i \geq 0$). The initial mean is then:

$
m = \sum_i f_i v_i
$

We now introduce the series $\omega_i$ which is defined using the parameter $\alpha$:

$
\omega_i(\alpha) = \alpha ^ {\left( \frac{2i}{N} - 1 \right)}
$

The term in the exponent covers the interval $[-1, 1]$. If $\alpha=0$, all $\omega_i$ will be $1$. For $\alpha \leq 1$ one has $\omega_i \leq \omega_{i+1}$ and for $\alpha \geq 1$ one has $\omega_i \geq \omega_{i+1}$. Hence, in one case more importance is put on lower values, and the opposite in the other case. The parameter $\alpha$ determines how strong the difference between the lower and higher values is.

We define a reweighting of the attribute distribution as:

$
f_i'(\alpha) = \frac{f_i \cdot \omega_i(\alpha)}{\sum_i f_i \cdot \omega_i(\alpha)}
$

Accordingly, a new mean can be calculated as:

$
m'(\alpha) = \sum_i f_i'(\alpha) v_i
$

The goal is now to find the value of $\alpha$ that moves $m'(\alpha)$ as close as possible to a desired target mean $\hat m$: 

$
\text{min}_\alpha \ |m'(\alpha) - \hat m|
$

This optimization can be performed using a simple bisection method.

In [15]:
# Prepare the calibration code
def calibrate_distribution(counts, values, target_mean, lower = 0.1, upper = 5.0):
    def objective(alpha, return_density = False):
        density = counts / np.sum(counts)

        weights = np.linspace(0, 2, len(density)) - 1.0
        weights = alpha ** weights

        density = density * weights
        density = density / np.sum(density)

        mean = np.sum(density * values)

        if not return_density:
            return mean - target_mean
        else:
            return density

    optimal_alpha = opt.bisect(objective, lower, upper)
    optimal_density = objective(optimal_alpha, True)
    return optimal_density, optimal_alpha

#### Household sizes

In [16]:
target_means = household_size_target_means

In [17]:
df_initial = df_census.groupby("household_size")["weight"].sum().reset_index()
df_initial = df_initial.sort_values(by = "household_size")

f = df_initial["household_size"] != NA
counts = df_initial.loc[f, "weight"].values
values = df_initial.loc[f, "household_size"].values
remainder = df_initial.loc[~f, "weight"].sum()

df_households = []

for scenario, target_mean in target_means.items():
    density, alpha = calibrate_distribution(counts, values, target_mean, upper = 5.0)

    updated_counts = density * np.sum(counts)
    updated_mean = np.sum(density * values)
    print("Scenario", scenario, "alpha=", alpha, "mean=", updated_mean, "target=", target_mean)

    df_households.append(pd.DataFrame({
        "scenario": [scenario] * (len(values) + 1),
        "household_size": list(values) + [NA],
        "weight": list(updated_counts) + [remainder]
    }))

df_households.insert(0, pd.DataFrame({
    "scenario": "Baseline",
    "household_size": list(values) + [NA],
    "weight": list(counts) + [remainder]
}))

df_households = pd.concat(df_households)

Scenario S1 alpha= 0.8371696790493163 mean= 3.093000000000332 target= 3.093
Scenario S2 alpha= 0.44457893962360234 mean= 2.9380000000005113 target= 2.938
Scenario S3 alpha= 2.579127134932719 mean= 3.401999999999923 target= 3.402
Scenario S4 alpha= 0.22445859518659292 mean= 2.783000000000728 target= 2.783


In [18]:
df_plot = df_households.copy()
df_plot["household_size"] = df_plot["household_size"].astype(str).replace({ str(NA): "NA" })

px.bar(
    df_plot, x = "household_size", y = "weight", color = "scenario", barmode = "group",
    title = "Household size: full absolute distribution after reweighting")

In [19]:
# Reduce the distribution size
df_reduced = df_households.copy()

df_reduced["household_size"] = df_reduced["household_size"]
df_reduced.loc[df_reduced["household_size"] > 6, "household_size"] = NA

df_reduced = df_reduced.groupby(["scenario", "household_size"])["weight"].sum().reset_index()
df_reduced = df_reduced.sort_values(by = "household_size")

In [20]:
df_plot = df_reduced.copy()
df_plot["household_size"] = df_plot["household_size"].astype(str).replace({ str(NA): "NA" })

px.bar(
    df_plot, x = "household_size", y = "weight", color = "scenario", barmode = "group",
    title = "Household size: reduced absolute distribution")

In [21]:
df_density = df_reduced.copy()
df_density = df_density.merge(df_density.groupby("scenario")["weight"].sum().reset_index(name = "total"))
df_density["density"] = df_density["weight"] / df_density["total"]

In [22]:
df_plot = df_density.copy()
df_plot["household_size"] = df_plot["household_size"].astype(str).replace({ str(NA): "NA" })

px.bar(
    df_plot, x = "household_size", y = "density", color = "scenario", barmode = "group",
    title = "Household size: reduced relative distribution")

In [23]:
df_household_size_density = df_density.copy()

#### Number of cars

In [24]:
target_means = number_of_cars_target_means

In [25]:
df_initial = df_census.groupby("number_of_cars")["weight"].sum().reset_index()
df_initial = df_initial.sort_values(by = "number_of_cars")

f = df_initial["number_of_cars"] != NA
counts = df_initial.loc[f, "weight"].values
values = df_initial.loc[f, "number_of_cars"].values
remainder = df_initial.loc[~f, "weight"].sum()

df_number_of_cars = []

for scenario, target_mean in target_means.items():
    density, alpha = calibrate_distribution(counts, values, target_mean, 0.01, 2.0)

    updated_counts = density * np.sum(counts)
    updated_mean = np.sum(density * values)
    print("Scenario", scenario, "alpha=", alpha, "mean=", updated_mean, "target=", target_mean)

    df_number_of_cars.append(pd.DataFrame({
        "scenario": [scenario] * (len(values) + 1),
        "number_of_cars": list(values) + [NA],
        "weight": list(updated_counts) + [remainder]
    }))

df_number_of_cars.insert(0, pd.DataFrame({
    "scenario": "Baseline",
    "number_of_cars": list(values) + [NA],
    "weight": list(counts) + [remainder]
}))

df_number_of_cars = pd.concat(df_number_of_cars)

Scenario S1 alpha= 1.1233096338853006 mean= 1.1349999999997842 target= 1.135
Scenario S2 alpha= 0.8877153097544669 mean= 1.0269999999993675 target= 1.027
Scenario S3 alpha= 0.8877153097544669 mean= 1.0269999999993675 target= 1.027
Scenario S4 alpha= 0.9999694965430445 mean= 1.0809999999999023 target= 1.081


In [26]:
df_plot = df_number_of_cars.copy()
df_plot["number_of_cars"] = df_plot["number_of_cars"].astype(str).replace({ str(NA): "NA" })

px.bar(
    df_plot, x = "number_of_cars", y = "weight", color = "scenario", barmode = "group",
    title = "Number of cars: full distribution after reweighting")

In [27]:
# Reduce the distribution size
df_reduced = df_number_of_cars.copy()

df_reduced["number_of_cars"] = df_reduced["number_of_cars"]
df_reduced.loc[df_reduced["number_of_cars"] > 6, "number_of_cars"] = NA

df_reduced = df_reduced.groupby(["scenario", "number_of_cars"])["weight"].sum().reset_index()
df_reduced = df_reduced.sort_values(by = "number_of_cars")

In [28]:
df_plot = df_reduced.copy()
df_plot["number_of_cars"] = df_plot["number_of_cars"].astype(str).replace({ str(NA): "NA" })

px.bar(
    df_plot, x = "number_of_cars", y = "weight", color = "scenario", barmode = "group",
    title = "Number of cars: reduced absolute distribution")

In [29]:
df_density = df_reduced.copy()
df_density = df_density.merge(df_density.groupby("scenario")["weight"].sum().reset_index(name = "total"))
df_density["density"] = df_density["weight"] / df_density["total"]

In [30]:
df_plot = df_density.copy()
df_plot["number_of_cars"] = df_plot["number_of_cars"].astype(str).replace({ str(NA): "NA" })

px.bar(
    df_plot, x = "number_of_cars", y = "density", color = "scenario", barmode = "group",
    title = "Number of cars: reduced relative distribution")

In [31]:
df_number_of_cars_density = df_density.copy()

#### Location type

In [32]:
df_initial = df_census.groupby("location_type")["weight"].sum().reset_index()
df_initial = df_initial.sort_values(by = "location_type")

df_location_type = []
for scenario, values in location_type_factors.items():
    df_scenario = df_initial.copy()
    
    for value, factor in values.items():
        f = df_scenario["location_type"] == value
        df_scenario.loc[f, "weight"] += df_scenario["weight"].sum() * factor
    
    df_scenario["scenario"] = scenario
    df_location_type.append(df_scenario)

df_location_type.insert(0, pd.DataFrame({
    "scenario": "Baseline",
    "location_type": df_initial["location_type"].values,
    "weight": df_initial["weight"].values
}))

df_location_type = pd.concat(df_location_type)

In [33]:
df_plot = df_location_type.copy()
df_plot["location_type"] = df_plot["location_type"].astype(str).replace({ str(NA): "NA" })

px.bar(
    df_plot, x = "location_type", y = "weight", color = "scenario", barmode = "group",
    title = "Location type: full distribution after reweighting")

In [34]:
df_density = df_location_type.copy()
df_density = df_density.merge(df_density.groupby("scenario")["weight"].sum().reset_index(name = "total"))
df_density["density"] = df_density["weight"] / df_density["total"]

In [35]:
df_plot = df_density.copy()
df_plot["location_type"] = df_plot["location_type"].astype(str).replace({ str(NA): "NA" })

px.bar(
    df_plot, x = "location_type", y = "density", color = "scenario", barmode = "group",
    title = "Location type: full relative distribution")

In [36]:
df_location_type_density = df_density.copy()

### Results

#### Household size

In [37]:
(df_household_size_density[["scenario", "household_size", "density"]].pivot(index = "household_size", columns = "scenario") * 100).round(2)

density                            
scenario       Baseline     S1     S2     S3     S4
household_size                                     
1                 16.06  16.68  18.95  12.96  21.52
2                 23.95  24.43  26.05  21.24  27.62
3                 18.32  18.36  18.38  17.87  18.20
4                 21.48  21.14  19.86  23.02  18.37
5                 11.33  10.95   9.66  13.35   8.35
6                  4.41   4.19   3.47   5.71   2.80
9999               4.45   4.24   3.63   5.85   3.14

In [38]:
df_household_size_density[[
    "household_size", "weight", "scenario"
]].to_parquet("{}/distribution_household_size.parquet".format(output_path), index = False)

#### Number of cars

In [39]:
(df_number_of_cars_density[["scenario", "number_of_cars", "density"]].pivot(index = "number_of_cars", columns = "scenario") * 100).round(2)

density                            
scenario       Baseline     S1     S2     S3     S4
number_of_cars                                     
0                 25.35  23.27  27.57  27.57  25.35
1                 44.29  43.93  44.48  44.48  44.29
2                 23.72  25.42  22.00  22.00  23.72
3                  4.79   5.55   4.11   4.11   4.79
9999               1.84   1.84   1.84   1.84   1.84

In [40]:
df_number_of_cars_density[[
    "number_of_cars", "weight", "scenario"
]].to_parquet("{}/distribution_number_of_cars.parquet".format(output_path), index = False)

#### Location type

In [41]:
(df_location_type_density[["scenario", "location_type", "density"]].pivot(index = "location_type", columns = "scenario") * 100).round(2)

density                            
scenario      Baseline     S1     S2     S3     S4
location_type                                     
1                29.50  29.50  33.55  33.55  25.54
2                15.30  15.30  15.32  15.32  15.32
3                22.53  22.53  18.40  18.40  26.41
9999             32.67  32.67  32.72  32.72  32.72

In [42]:
df_location_type_density[[
    "location_type", "weight", "scenario"
]].to_parquet("{}/distribution_location_type.parquet".format(output_path), index = False)